# Creating airfoil data for various Reynolds numbers

When analyzing propellors using blade element momentum theory we find that the various elements see a large variety in speeds and therefore reynolds number. Thus to keep the fidelity of the model within acceptable bounds it is required to know the properties of the airfoil at a wide range of Reynolds number.

In order to so we can use tools such as xfoil. Xfoil is a command line tool developed by Mark Drela also known from AVL. It lets you analyze 2d airfoils using only a small set of commands. The software is open source and can be downloaded from [xfoil](https://web.mit.edu/drela/Public/web/xfoil/). Full documentation can be found in the same link as well.

## Workflow for retrieving polar data

Opening the xfoil executable will give you the screen as shown hereunder. It is a command line interface (CLI) tool thus you will have to type the commands. You can always see which commands are available at anytime by entering `?` and pressing enter. The following procedure should then be followed to retrieve the polar data.

1. First we load in an airfoil, xfoil makes it very easy to load in NACA airfoils using the command `naca` with any 4 or 5 digit code so e.g `naca 4412`. However, any airfoil can be loaded in using `load example/path` where example/path points to  a coordinate file.

2. Once an airfoil has been chosen you go to operation mode by entering `oper`.

3. Once in operation mode it is advised to increase the maximum amount of iterations for convergence sakes this can be done through `iter 40`. Where the numbers represents the maximuma amount of iterations.

4. Then the viscous model should be enabled using `visc 1000000`. Where the integer represents the reynolds number, this is just an initialization and can still be changed afterwards using the command `r integer` to the desired reynolds numbers.

5. To then write a text file with  polar data for the current configured Reynolds number we enter the `pacc` command. Xfoil will then ask for a save path to write the file to. I recommend writing it to your downloads folder or another short path. As the path handling is not as robust and xfoil will fail to write the files with longer paths. E.g if you want to write a file named `4412_re3000000.txt` to your downloads folder you will enter the following: `C:\Users\JohnDoe\Downloads\4412_re3000000.txt`. The program will then ask for a dump path, this can safely be ignored by filling in a blank line. 

6. Then xfoil can compute the result of a range of angle of attack by using `aseq start_aoa end_alpha step_alpha` where `start_aoa` represents the starting angle of attack, `end_alpha` the ending angle of attack and `step_alpha` the step in the angle of attack.

7. Whilst doing so it should have plotted the cp distribution as a pop up. You will now find a file in the specified directory. To now swithc to a different reynolds number we must first stop writing in the current file by typing `pacc` again. You can now change the reynolds number by using eg `r 20000000`.


# Automating the process using Python

As seen from the previous explanation it would be a lot of work to do create all the necessary data for a wide range of Reynolds numbers. Hence we will automate this process using the `subprocess` module in Python which can manage subprocesses where the Python script itself is the main process. The script here under allows you to do so. The code is specifically left in Mardown as it requires a path to the xfoil executable thus you will have to copy the code below. However, a dataset is already embedded in the repository in the `examples\data` folder. Additionally, the script can also be found in the `examples\automation_airfoil_data.py`. Some notes about running the script can be found below.

```python
import subprocess
import os
import numpy as np


# Path to the XFOIL executable
xfoil_path = r'C:\Users\Johndoe\Xfoil699src\xfoil.exe'
save_path = r'C:\Users\Johndoe\Downloads\data' # Path can not be too long as it wil fail at some point
naca_airfoil = "4412"
max_iter = "40"

# Reynolds range
start_reyn = 2e6
ending_reyn = 50e6
step_reyn = 1e6

#AoA range
start_alpha = "-5"
end_alpha = "16"
step_alpha = "0.2"

# XFOIL  start up commands
startup_commands = [
    'naca ' + naca_airfoil,
    'oper',
    'iter' + max_iter,
    "visc 1000000"
]

# Run XFOIL
process = subprocess.Popen(xfoil_path, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

for command in startup_commands:
    process.stdin.write(command + '\n')  
    process.stdin.flush()  

for  Reyn in  np.arange(start_reyn, ending_reyn, step_reyn):
    Reyn = int(Reyn)
    process.stdin.write("r"+ str(Reyn)  + '\n')  
    process.stdin.write("pacc"+ '\n')  
    process.stdin.write(os.path.join(save_path, "airfoil_" + str(Reyn)) +  ".txt" + '\n')  
    process.stdin.write('\n')  
    process.stdin.write(f'aseq {start_alpha} {end_alpha} {step_alpha}' + '\n')  
    process.stdin.write('pacc' + '\n')  
    process.stdin.flush()

output, error  = process.communicate()
```

## Operation of the Python script

1. Xfoil tends to crash at different Reynolds number. Thus sometimes you have to restart the script starting from the last successful Reynolds numbers therefore you will have to alter the variable `start_reyn`.
2. Not all angle of attack converge properly to give the best possible results do the following:
    - Increase the maximum iterations
    - Decrease the step in angle of attack
    - Keep your airfoil open at the trailing, this tends to help the convergence
    